### np.partition函数
1. [文章详细介绍](https://mp.weixin.qq.com/s?__biz=MzU3MDkzNjk1OQ==&mid=2247491247&idx=1&sn=91ced6a7ef1c9bd304cf15ffca5fe879&chksm=fce685bfcb910ca908cbcbb25c194bc9821596e8c78f06c8cacb8f1cc6884f2f490df7186f59&token=1553435301&lang=zh_CN#rd)

In [2]:
import numpy as np 

`np.partition`是对一个向量、沿着一个维度方向、按照大小对数据进行分堆，分成了两堆。
1. 在`kth`前面的这堆数值，都是这个向量里面比较小的群体们。
2. 在`kth`后面的这堆数值，都是这个向量里面比较大的群体们。

In [3]:
data = np.array([232, 564, 278, 3, 2, 1, -1, -10, -30, -40])
np.partition(data, kth=4)

array([-10, -30, -40,  -1,   1,   2, 564,   3, 278, 232])

`np.partition(data, kth=4)`的意思就是：
1. `np.partition`对`data`说：“你们班，现在给我从左向右站好！我不需要你们完全从低到高排序好，我只要左边4个是你们里面最小的就行”
2. 然后`data`就找到班里最小的4个同学：`-1`, `-10`, `-30`, `-40`，说：“你们几个赶快给我站到左边，不需要你们几个再排序了，怎么快怎么来”
3. `-1`, `-10`, `-30`, `-40`听到`data`指令后，马上跑到左边站好。
4. 剩下的没有被指出来的，想怎么站都无所谓

1. `np.partition`功能和`np.argpartition`功能是一样的，只不过`np.argpartition`返回的是序号

In [4]:
np.argpartition(data, kth=4)

array([7, 8, 9, 6, 5, 4, 1, 3, 2, 0])

1. `np.argpartition`对`data`说:“把你们班最差的几个人学号放在前4个坑，剩下人随便填上就行了。
2. `data`对`-1`, `-10`, `-30`, `-40`说，你们几个差学生也别出来了，就对我说你们学号多少，我来填上，然后`-1`说我是6号，`-10`说我说7号，`-30`说我是8号，`-40`说我是9号。
3. 然后剩下的人的编号，`data`敷衍了事，随便写上了。

那么我们要想找到`data`的最小的4个数字，其实非常简单。两个方法

In [17]:
# way 1
np.partition(data, kth=4)[:4]

# way 2
data[np.argpartition(data, kth=4)[:4]]

array([-10, -30, -40,  -1])

那么问题来了，我想找到`data`最大的3位数怎么办？
1. 一般第k个，我们在python里面都是使用k为正数，也就是从左向右数第k个。
2. 在python取后k个，其实我们都是知道的，就是使用`-k`这个方法。

那么这个方法其实在这里也是适用的，下面就是解决方法。就不过多做解释了。（注意返回的结果的后三个数值）

In [19]:
np.partition(data, kth=-3)# 返回的是具体的值

array([  1, -40, -30, -10,  -1,   2,   3, 232, 278, 564])

In [20]:
np.argpartition(data, kth=-3) # 返回的是值对应的序号

array([5, 9, 8, 7, 6, 4, 3, 0, 2, 1])

我要是取后3个序号，其实就是top3的值了

In [5]:
# way1
np.partition(data, kth=-3)[-3:]

# way 2
data[np.argpartition(data, kth=-3)[-3:]]

array([232, 278, 564])

上面的`data`只是一维的，对于二维及更高维度的数据同样适用。
这里要注意一个小细节：
1. 假设一个数组的shape是: (m,n,z)
那么`axis=1`的方向其实就是沿着第二个也就是`n`这个方向。希望可以帮助读者分清楚

### 实际问题
我是一个研究疾病的研究生，手上有个数据：
1. 有一系列数据：其中有425个`疾病名称`，有13426个`症状`
2. 还有一个`疾病名称`和`症状`的`权重matrix`，矩阵的shape为`425 x 13426`

需要解决的问题是：
我需要按照`权重matrix`找到每个`疾病名称`前10个最相关的`症状`，并且记录下来。


#### 解析
1. 这里需要处理的数组变成了二维数组，找top10（没让你排序，只要找到就行了），并且记录下来。
2. 这里使用`np.argpartition`可以一次性将所有的topk找出来，大大的提高了计算量

In [7]:
import numpy as np
import pandas as pd
from tqdm import tqdm
# generate sample data
n_features = 13426
n_disease = 425
features = [f"feature_{i}" for i in range(n_features)]
disease = [f"disease_{i}" for i in range(n_disease)]
weights = np.random.random((n_disease, n_features))
#function
def getdata(top_k: int) -> pd.DataFrame:
    index = np.argpartition(weights, -top_k, axis=1)[:, -top_k:]

    def slice_data(i):
        temp_data = pd.DataFrame({
            'features': np.array(features)[index[i, :]]})
        temp_data['disease'] = disease[i]
        temp_data['weights'] = weights[i, index[i, :]]
        return temp_data

    res = pd.concat([slice_data(i) for i in tqdm(range(weights.shape[0]))]).reset_index(drop=True)
    return res


final_data = getdata(top_k=3) # 这里只是找top3的，要是想找top10的，修改数值就行了
final_data.shape
final_data.head(10)

100%|██████████| 425/425 [00:01<00:00, 254.92it/s]


,features,disease,weights
0,feature_12018,disease_0,0.999897
1,feature_8510,disease_0,0.999966
2,feature_268,disease_0,0.999974
3,feature_4504,disease_1,0.999844
4,feature_5604,disease_1,0.999965
5,feature_777,disease_1,0.999873
6,feature_1221,disease_2,0.999905
7,feature_7040,disease_2,0.999963
8,feature_9973,disease_2,0.999927
9,feature_3787,disease_3,0.999900
